# Testing ⏭ GreedLR Scheduler for 🤗 Transformers

## 1. Translation

### Need to reinstall from source to register changes

(may need to restart kernel)

In [2]:
%pip install -r translation/requirements.txt
%pip install -e ~/transformers/ #Or wherever you downloaded this source 


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Obtaining file:///root/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.27.0.dev1-0.editable-py3-none-any.whl size=34577 sha256=7b8b1ba0ccc4a02285bca1643653e84808dd8206d855780c2c9dd20f55bdfa88
  Stored in directory: /root/transformers/examples/pytorch/pip-ephem-wheel-cache-lf2pj9mo/wheels/c6/d4/24/fe65e904e9dddeaa02668d56c7a9bd848371752e768d6bb33b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.0.dev1
    Uninstalling transformers-4.27.0.dev1:
      Succ

Add test folders to PATH

In [3]:
import sys
import os
SRC_DIRS = [
    os.path.join('./', dirname) for dirname in [
        "text-generation",
        "text-classification",
        "token-classification",
        "language-modeling",
        "multiple-choice",
        "question-answering",
        "summarization",
        "translation",
        "image-classification",
        "speech-recognition",
        "audio-classification",
        "speech-pretraining",
        "image-pretraining",
        "semantic-segmentation",
    ]
]
sys.path.extend(SRC_DIRS)

In [4]:
from translation import run_translation
import argparse
import json
import logging
import os
import sys
from unittest.mock import patch

import torch

from transformers import ViTMAEForPreTraining, Wav2Vec2ForPreTraining
from transformers.testing_utils import CaptureLogger, TestCasePlus, get_gpu_count, slow, torch_device
from transformers.utils import is_apex_available
from utils import *

def get_results(output_dir):
    results = {}
    path = os.path.join(output_dir, "all_results.json")
    if os.path.exists(path):
        with open(path, "r") as f:
            results = json.load(f)
    else:
        raise ValueError(f"can't find {path}")
    return results

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# %pip install sacremoses

### Default AdamW_HF with LambdaLR (linear, default)

In [ ]:
tmp_dir = get_auto_remove_tmp_dir()
testargs = f"""
    run_translation.py
    --model_name_or_path facebook/wmt19-de-en
    --source_lang de
    --target_lang en
    --dataset_name wmt16
    --dataset_config_name de-en
    --output_dir {tmp_dir}
    --overwrite_output_dir
    --max_steps=5000
    --do_train
    --learning_rate=1e-4
    --per_device_train_batch_size=4
    --per_device_eval_batch_size=4
    --predict_with_generate
    --save_strategy no
    --logging_steps 10
    --seed 42
    --lr_scheduler_type cosine_with_restarts
    --optim adafactor
""".split()

with patch.object(sys, "argv", testargs):
    run_translation.main()
    result = get_results(tmp_dir)
    # print(result["eval_bleu"]>30)


03/18/2023 04:57:54 - WARNING - translation.run_translation - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/18/2023 04:57:54 - INFO - translation.run_translation - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
factor=0.99,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wr

100%|██████████| 3/3 [00:00<00:00, 61.53it/s]
[INFO|configuration_utils.py:668] 2023-03-18 04:57:54,580 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--wmt19-de-en/snapshots/f489ab7724347fcf69cf7cf057fe5a4d06920d64/config.json
[INFO|configuration_utils.py:720] 2023-03-18 04:57:54,584 >> Model config FSMTConfig {
  "_commit_hash": "f489ab7724347fcf69cf7cf057fe5a4d06920d64",
  "_name_or_path": "facebook/wmt19-de-en",
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "FSMTForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "d_model": 1024,
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": false,
    "architectures": null,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 2,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_

03/18/2023 04:57:58 - WARNING - datasets.arrow_dataset - Loading cached processed dataset at /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227/cache-23204a732454da9c.arrow


[INFO|trainer.py:543] 2023-03-18 04:58:00,156 >> max_steps is given, it will override any value given in num_train_epochs
[INFO|trainer.py:1745] 2023-03-18 04:58:00,174 >> ***** Running training *****
[INFO|trainer.py:1746] 2023-03-18 04:58:00,175 >>   Num examples = 4548885
[INFO|trainer.py:1747] 2023-03-18 04:58:00,175 >>   Num Epochs = 1
[INFO|trainer.py:1748] 2023-03-18 04:58:00,176 >>   Instantaneous batch size per device = 4
[INFO|trainer.py:1749] 2023-03-18 04:58:00,176 >>   Total train batch size (w. parallel, distributed & accumulation) = 4
[INFO|trainer.py:1750] 2023-03-18 04:58:00,176 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:1751] 2023-03-18 04:58:00,177 >>   Total optimization steps = 5000
[INFO|trainer.py:1752] 2023-03-18 04:58:00,178 >>   Number of trainable parameters = 312778752


[2023-03-18 04:58:00.322: W smdistributed/modelparallel/torch/nn/predefined_hooks.py:78] Found unsupported HuggingFace version 4.27.0.dev0 for automated tensor parallelism. HuggingFace modules will not be automatically distributed. You can use smp.tp_register_with_module API to register desired modules for tensor parallelism, or directly instantiate an smp.nn.DistributedModule. Supported HuggingFace transformers versions for automated tensor parallelism: ['4.17.0', '4.20.1', '4.21.0']
[2023-03-18 04:58:00.351 pytorch-1-13-gpu-py-ml-g5-16xlarge-bff23aa7fcca0c0ba1c19f2e4a3e:36376 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-03-18 04:58:00.376 pytorch-1-13-gpu-py-ml-g5-16xlarge-bff23aa7fcca0c0ba1c19f2e4a3e:36376 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


Step,Training Loss
10,5.626900
20,4.229100
30,3.325300
40,2.352900
50,1.939700
60,1.272400
70,0.949700
80,1.000900
90,1.310000
100,1.334500


In [ ]:
import json

alllogs = json.load(open(f'{tmp_dir}/trainer_state.json'))
d1 = [ (l['step'], l['learning_rate'], l['loss']) for l in alllogs['log_history'][:-1] ]

!rm -r {tmp_dir}

### With GreedyLR

In [ ]:
tmp_dir = get_auto_remove_tmp_dir()
testargs = f"""
    run_translation.py
    --model_name_or_path facebook/wmt19-de-en
    --source_lang de
    --target_lang en
    --dataset_name wmt16
    --dataset_config_name de-en
    --output_dir {tmp_dir}
    --overwrite_output_dir
    --max_steps=5000
    --warmup_steps=0
    --do_train
    --learning_rate=1e-4
    --per_device_train_batch_size=4
    --per_device_eval_batch_size=4
    --predict_with_generate
    --lr_scheduler_type greedy
    --save_strategy no
    --logging_steps 10
    --min_lr=1e-5
    --smooth True
    --patience 10
    --factor 0.95
    --seed 42
    --optim adafactor
""".split()

with patch.object(sys, "argv", testargs):
    run_translation.main()
    result = get_results(tmp_dir)
    # print(result["eval_bleu"]>30)


In [ ]:
alllogs = json.load(open(f'{tmp_dir}/trainer_state.json'))
d2 = [ (l['step'], l['learning_rate'], l['loss']) for l in alllogs['log_history'][:-1] ]

!rm -r {tmp_dir}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
x1 = np.array(d1)[:,0]
y1 = np.array(d1)[:,1]

x2 = np.array(d2)[:,0] 
y2 = np.array(d2)[:,1]

fig = plt.figure()
ax = fig.add_subplot(111)
line1, = ax.plot(x1, y1, 'b-', label='baseline')
line2, = ax.plot(x2, y2, 'r-', label='GreedyLR')
plt.legend()
plt.title('LRs')

In [ ]:
x1 = np.array(d1)[:,0]
y1 = np.array(d1)[:,2]

x2 = np.array(d2)[:,0] 
y2 = np.array(d2)[:,2]

fig = plt.figure()
ax = fig.add_subplot(111)
line1, = ax.plot(x1, y1, 'b-',label='baseline')
line2, = ax.plot(x2, y2, 'r-',label='GreedyLR')
plt.legend()
plt.title('Loss')

In [ ]:
d1[-1][-1] - d2[-1][-1]

## 2. Question Answering

In [ ]:
import run_qa as run_squad
import argparse
import json
import logging
import os
import sys
from unittest.mock import patch

import torch

from transformers import ViTMAEForPreTraining, Wav2Vec2ForPreTraining
from transformers.testing_utils import CaptureLogger, TestCasePlus, get_gpu_count, slow, torch_device
from transformers.utils import is_apex_available
from utils import *

def get_results(output_dir):
    results = {}
    path = os.path.join(output_dir, "all_results.json")
    if os.path.exists(path):
        with open(path, "r") as f:
            results = json.load(f)
    else:
        raise ValueError(f"can't find {path}")
    return results

### Default ADAMHF / Adafactor

In [ ]:
tmp_dir = get_auto_remove_tmp_dir()
testargs = f"""
    run_qa.py
    --model_name_or_path xlm-roberta-base
    --version_2_with_negative
    --dataset_name squad
    --output_dir {tmp_dir}
    --overwrite_output_dir
    --max_steps=5000
    --do_train
    --learning_rate=1e-4
    --per_device_train_batch_size=2
    --per_device_eval_batch_size=1
    --save_strategy no
    --logging_steps 10
    --seed 42
    --lr_scheduler_type cosine_with_restarts
    --optim adafactor
""".split()

    # --lr_scheduler_type cosine_with_restarts
    # --optim adafactor

with patch.object(sys, "argv", testargs):
    run_squad.main()
    result = get_results(tmp_dir)

In [ ]:
import json

alllogs = json.load(open(f'{tmp_dir}/trainer_state.json'))
d1 = [ (l['step'], l['learning_rate'], l['loss']) for l in alllogs['log_history'][:-1] ]

!rm -r {tmp_dir}

### Greedy LR

In [ ]:
tmp_dir = get_auto_remove_tmp_dir()
testargs = f"""
    run_qa.py
    --model_name_or_path xlm-roberta-base
    --version_2_with_negative
    --dataset_name squad
    --output_dir {tmp_dir}
    --overwrite_output_dir
    --max_steps=5000
    --warmup_steps=0
    --do_train
    --learning_rate=1e-4
    --per_device_train_batch_size=4
    --per_device_eval_batch_size=2
    --lr_scheduler_type greedy
    --save_strategy no
    --logging_steps 10
    --min_lr=1e-5
    --smooth True
    --patience 10
    --factor 0.95
    --seed 42
    --optim adafactor
""".split()

with patch.object(sys, "argv", testargs):
    run_squad.main()
    result = get_results(tmp_dir)

In [ ]:
alllogs = json.load(open(f'{tmp_dir}/trainer_state.json'))
d2 = [ (l['step'], l['learning_rate'], l['loss']) for l in alllogs['log_history'][:-1] ]

!rm -r {tmp_dir}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
x1 = np.array(d1)[:,0]
y1 = np.array(d1)[:,1]

x2 = np.array(d2)[:,0] 
y2 = np.array(d2)[:,1]

fig = plt.figure()
ax = fig.add_subplot(111)
line1, = ax.plot(x1, y1, 'b-', label='baseline')
line2, = ax.plot(x2, y2, 'r-', label='GreedyLR')
plt.legend()
plt.title('LRs')

In [ ]:
x1 = np.array(d1)[:,0]
y1 = np.array(d1)[:,2]

x2 = np.array(d2)[:,0]
y2 = np.array(d2)[:,2]

fig = plt.figure()
ax = fig.add_subplot(111)
line1, = ax.plot(x1, y1, 'b-', label='baseline')
line2, = ax.plot(x2, y2, 'r-', label='GreedyLR')
plt.legend()
plt.title('Loss')

In [ ]:
d1[-1][-1] - d2[-1][-1]

## Image Classification

In [ ]:
import run_image_classification
import argparse
import json
import logging
import os
import sys
from unittest.mock import patch

import torch

from transformers import ViTMAEForPreTraining, Wav2Vec2ForPreTraining
from transformers.testing_utils import CaptureLogger, TestCasePlus, get_gpu_count, slow, torch_device
from transformers.utils import is_apex_available
from utils import *

def get_results(output_dir):
    results = {}
    path = os.path.join(output_dir, "all_results.json")
    if os.path.exists(path):
        with open(path, "r") as f:
            results = json.load(f)
    else:
        raise ValueError(f"can't find {path}")
    return results

In [ ]:
tmp_dir = get_auto_remove_tmp_dir()
testargs = f"""
    run_image_classification.py
    --output_dir {tmp_dir}
    --model_name_or_path microsoft/resnet-152
    --dataset_name cifar100
    --do_train
    --learning_rate 1e-4
    --per_device_train_batch_size 64
    --remove_unused_columns False
    --overwrite_output_dir True
    --metric_for_best_model accuracy
    --max_steps 1000
    --train_val_split 0.2
    --save_strategy no
    --logging_steps 10
    --ignore_mismatched_sizes True
    --seed 42
    --lr_scheduler_type cosine_with_restarts
    --optim adafactor
""".split()

with patch.object(sys, "argv", testargs):
    run_image_classification.main()
    result = get_results(tmp_dir)


In [ ]:
import json

alllogs = json.load(open(f'{tmp_dir}/trainer_state.json'))
d1 = [ (l['step'], l['learning_rate'], l['loss']) for l in alllogs['log_history'][:-1] ]

!rm -r {tmp_dir}

!rm -r pymp*
!rm -r ./tmp*

In [ ]:
tmp_dir = get_auto_remove_tmp_dir()
testargs = f"""
    run_image_classification.py
    --output_dir {tmp_dir}
    --model_name_or_path microsoft/resnet-152
    --dataset_name cifar100
    --do_train
    --learning_rate 1e-4
    --per_device_train_batch_size 64
    --remove_unused_columns False
    --overwrite_output_dir True
    --metric_for_best_model accuracy
    --max_steps 1000
    --train_val_split 0.2
    --save_strategy no
    --logging_steps 10
    --ignore_mismatched_sizes True
    --lr_scheduler_type greedy
    --logging_steps 10
    --min_lr=1e-5
    --smooth True
    --patience 10
    --factor 0.95
    --seed 42
    --optim adafactor
""".split()

with patch.object(sys, "argv", testargs):
    run_image_classification.main()
    result = get_results(tmp_dir)

In [ ]:
alllogs = json.load(open(f'{tmp_dir}/trainer_state.json'))
d2 = [ (l['step'], l['learning_rate'], l['loss']) for l in alllogs['log_history'][:-1] ]

!rm -r {tmp_dir}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
x1 = np.array(d1)[:,0]
y1 = np.array(d1)[:,1]

x2 = np.array(d2)[:,0] 
y2 = np.array(d2)[:,1]

fig = plt.figure()
ax = fig.add_subplot(111)
line1, = ax.plot(x1, y1, 'b-', label='baseline')
line2, = ax.plot(x2, y2, 'r-', label='GreedyLR')
plt.legend()
plt.title('LRs')

In [ ]:
x1 = np.array(d1)[:,0]
y1 = np.array(d1)[:,2]

x2 = np.array(d2)[:,0]
y2 = np.array(d2)[:,2]

fig = plt.figure()
ax = fig.add_subplot(111)
line1, = ax.plot(x1, y1, 'b-', label='baseline')
line2, = ax.plot(x2, y2, 'r-', label='GreedyLR')
plt.legend()
plt.title('Loss')

In [ ]:
d1[-1][-1] - d2[-1][-1]